<center>

# $\textbf{Unemployment}$

<center>

### $\textbf{Code}$

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, lit, array, struct

In [30]:
spark = SparkSession.builder.appName('Unemployment').master("local").enableHiveSupport().getOrCreate()
spark

In [31]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("Files/Unemployment2.csv", format="csv", sep=",", inferschema="true", header="true")

In [32]:
# Remove columns Country Code and Indicator Code
df.drop("Country Code", "Indicator Code", "Indicator Name")

DataFrame[Country Name: string, 1960: string, 1961: string, 1962: string, 1963: string, 1964: string, 1965: string, 1966: string, 1967: string, 1968: string, 1969: string, 1970: string, 1971: string, 1972: string, 1973: string, 1974: string, 1975: string, 1976: string, 1977: string, 1978: string, 1979: string, 1980: string, 1981: string, 1982: string, 1983: string, 1984: string, 1985: string, 1986: string, 1987: string, 1988: string, 1989: string, 1990: string, 1991: double, 1992: double, 1993: double, 1994: double, 1995: double, 1996: double, 1997: double, 1998: double, 1999: double, 2000: double, 2001: double, 2002: double, 2003: double, 2004: double, 2005: double, 2006: double, 2007: double, 2008: double, 2009: double, 2010: double, 2011: double, 2012: double, 2013: double, 2014: double, 2015: double, 2016: double, 2017: double, 2018: double, 2019: double, 2020: double, 2021: double, 2022: double]

In [33]:
# Rename the column
df = df.withColumnRenamed("Country Name","country")

In [34]:
# Assuming df is properly defined DataFrame
df = df.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("unemployment")) 
    for year in range(2000, 2023)
])).alias("data")).selectExpr("country", "data.year", "data.unemployment")

df = df.orderBy("country")

In [35]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("FilesParquet/Unemployment.parquet" )
spark.read.parquet("FilesParquet/Unemployment.parquet").show()
spark.stop()

+-----------+----+------------+
|    country|year|unemployment|
+-----------+----+------------+
|Afghanistan|2000|       7.942|
|Afghanistan|2001|       7.942|
|Afghanistan|2002|       7.935|
|Afghanistan|2003|       7.929|
|Afghanistan|2004|       7.926|
|Afghanistan|2005|       7.925|
|Afghanistan|2006|       7.925|
|Afghanistan|2007|       7.924|
|Afghanistan|2008|       7.928|
|Afghanistan|2009|       7.923|
|Afghanistan|2010|       7.921|
|Afghanistan|2011|       7.918|
|Afghanistan|2012|       7.914|
|Afghanistan|2013|       7.914|
|Afghanistan|2014|        7.91|
|Afghanistan|2015|       9.002|
|Afghanistan|2016|      10.092|
|Afghanistan|2017|       11.18|
|Afghanistan|2018|      11.131|
|Afghanistan|2019|      11.082|
+-----------+----+------------+
only showing top 20 rows

